## Connect to Google SQL Database (PostgreSQL)

In [1]:
## use conda env 'dsci551-project'

## libraries for connecting to PostgreSQL
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

import pandas as pd 
import numpy as np 
import sys
import datetime

# from GoogleNews import GoogleNews
# from newspaper import Article
import functools 
import operator
import time

In [2]:
conn = psycopg2.connect(
    user = "postgres",
    password = "BHeBq2NKxkxal4fq",
    host = "127.0.0.1",
    port = "5432",
    database = "news"
)

In [3]:
cur = conn.cursor()

In [4]:
## Print connection properties
print(conn.get_dsn_parameters(), "\n")

{'user': 'postgres', 'dbname': 'news', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [5]:
## Print postgres version
cur.execute("SELECT version();")
record = cur.fetchone()
print("Connected to ", record, "\n")

Connected to  ('PostgreSQL 12.3 on x86_64-pc-linux-gnu, compiled by Debian clang version 10.0.1 , 64-bit',) 



In [6]:
## Import a df and insert it into the db
df = pd.read_csv('data/df_merged.csv')
df.head()

,title,media,date,desc,link,img,Symbol
0,Turn Your Side Hustle into a Booming Startup,Street Fight,"Jan 29, 2020",My co-founders and I grew the platform to 3M+ ...,https://streetfightmag.com/2020/01/29/turn-you...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM
1,Security Innovation enhances cybersecurity tra...,NaN,"Jan 23, 2020","After finishing 2019 with 3M+ licensed users, ...",https://www.globenewswire.com/news-release/202...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM
2,"These ""Three Kings"" hold the record of the Fas...",allkpop,"Jan 10, 2020",#ShadowComebackTrailer featuring @BTS_twt's SU...,https://www.allkpop.com/article/2020/01/these-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM
3,Fast-Growing Drizly Joins With Leading Texas R...,PRNewswire,"Jan 13, 2020",... delivery to El Paso and Corpus Christi for...,https://www.prnewswire.com/news-releases/fast-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM
4,Roker Roundup: Reporter suggests that Blackpoo...,Roker Report,"Jan 2, 2020",The striker has only scored once in the league...,https://rokerreport.sbnation.com/2020/1/2/2104...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM


## Fixing problems with date format
(messed up date columns) and getting it to right daettime format for postgres

In [7]:
## fix weird formatted date strings
df['date'] = df['date'].str.replace(u'.*\xa0·\xa0', '')

In [8]:
## convert date col to datetime format
df['dt'] = pd.to_datetime(df['date'],format='%b %d, %Y',errors='coerce').dt.strftime('%Y-%m-%d')

In [9]:
## replace NaN values with today's date
df['dt'] = df['dt'].replace(np.nan, str([datetime.date.today()][0]))

In [10]:
## check if any NaNs left
# test = df[df['dt'].isnull()]
# test

In [11]:
# df.head(100)

In [12]:
## fix single quotes before isnerting into postgres
df['title'] = df['title'].str.replace("'", "''")
df['media'] = df['media'].str.replace("'","''")
df['desc'] = df['desc'].str.replace("'","''")

In [13]:
df.head(10)

,title,media,date,desc,link,img,Symbol,dt
0,Turn Your Side Hustle into a Booming Startup,Street Fight,"Jan 29, 2020",My co-founders and I grew the platform to 3M+ ...,https://streetfightmag.com/2020/01/29/turn-you...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM,2020-01-29
1,Security Innovation enhances cybersecurity tra...,NaN,"Jan 23, 2020","After finishing 2019 with 3M+ licensed users, ...",https://www.globenewswire.com/news-release/202...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM,2020-01-23
2,"These ""Three Kings"" hold the record of the Fas...",allkpop,"Jan 10, 2020",#ShadowComebackTrailer featuring @BTS_twt''s S...,https://www.allkpop.com/article/2020/01/these-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM,2020-01-10
3,Fast-Growing Drizly Joins With Leading Texas R...,PRNewswire,"Jan 13, 2020",... delivery to El Paso and Corpus Christi for...,https://www.prnewswire.com/news-releases/fast-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM,2020-01-13
4,Roker Roundup: Reporter suggests that Blackpoo...,Roker Report,"Jan 2, 2020",The striker has only scored once in the league...,https://rokerreport.sbnation.com/2020/1/2/2104...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM,2020-01-02
5,Spectacular Houses On The Market In London Jan...,Luxury London,"Jan 15, 2020",... to mention the striking vaulted chamber th...,https://luxurylondon.co.uk/house/property/lond...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM,2020-01-15
6,Top 10 Automotive Developments to Watch,Ward''s Auto,"Jan 6, 2020","Ride sharing is for real, and those dealers wh...",https://www.wardsauto.com/dealers/top-10-autom...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM,2020-01-06
7,Morien Resources Corp.: Production Inflection ...,Seeking Alpha,"Jan 10, 2020",Morien has required a lot of patience as inves...,https://seekingalpha.com/article/4316213-morie...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM,2020-01-10
8,Report: Matt Rhule considering LSU''s Joe Brad...,Panthers Wire,"Jan 11, 2020","... has agreed to a new contract, sources tell...",https://pantherswire.usatoday.com/2020/01/11/p...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM,2020-01-11
9,"Cardinals, Rockies Have Reportedly Engaged In ...",MLB Trade Rumors,"Jan 12, 2020","Despite their small market size, the Cardinals...",https://www.mlbtraderumors.com/2020/01/cardina...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",MMM,2020-01-12


In [14]:
for row in df.iterrows():

    idx = row[0]
    title = row[1][0]
    media = row[1][1]
    descrip = row[1][3]
    link = row[1][4]
    symbol = row[1][6]
    dt = row[1][7]

    try: 
        cur.execute(f"INSERT INTO news (news_id,symbol,title,descrip,media,date,link) VALUES (DEFAULT,'{symbol}','{title}', '{descrip}','{media}', '{dt}', '{link}');")

    except Exception as e:
        print(e)
        continue
    

In [15]:
try:
    cur.execute("SELECT * FROM news;")
    # record = cur.fetchall()
    record = cur.fetchone()
    print(record)
except Exception as e:
    print(e)

(202, 'MMM', 'Turn Your Side Hustle into a Booming Startup', "My co-founders and I grew the platform to 3M+ users and 10K sign-ups per day just three years after launch. We learned a lot along the way — here's what ...", 'Street Fight', datetime.date(2020, 1, 29), 'https://streetfightmag.com/2020/01/29/turn-your-side-hustle-into-a-booming-startup/')


In [16]:
## commit the changes to the db
conn.commit()

In [17]:
## close the connection
cur.close()
conn.close()
print("postgresql connection closed")

postgresql connection closed
